In [19]:
from Indicator import Indicator 
import random
import glob
import numpy as np
import pandas as pd

random.seed(0)

datalist = [n for n in glob.glob('Dataset/*.csv')]
indicator = Indicator()

In [23]:
def initial_population(n):#number Population
    pop = []
    for i in range(n):
        pop.append([[random.randint(1,60),random.randint(1,60),random.randint(1,60)],0])
    return pop

def cal_fitness(pop):
    calpop = []
    for cm in pop:
        fitness = 0
        df = pd.read_csv(datalist[0],index_col = False)
        df = indicator.MACD(df,cm[0][0],cm[0][1],cm[0][2])
        b = df.loc[(df['MACD']>df['SIGNAL LINE'])&(df['MACD'].shift()<df['SIGNAL LINE'].shift())]['Close']
        bd = df.loc[(df['MACD']>df['SIGNAL LINE'])&(df['MACD'].shift()<df['SIGNAL LINE'].shift())].index
        s = df.loc[(df['MACD'] < df['SIGNAL LINE'])  & (df['MACD'].shift()>df['SIGNAL LINE'].shift())]['Close']
        sd = df.loc[(df['MACD'] < df['SIGNAL LINE'])  & (df['MACD'].shift()>df['SIGNAL LINE'].shift())].index
        b,bd,s,sd = b.tolist(),bd.tolist(),s.tolist(),sd.tolist()
        
        money = 10000
      
        if len(b)==0 or len(s)==0:
            calpop.append([cm[0],0])
        else:
            if bd[0]>sd[0]:
                s.pop(0)
                sd.pop(0)   
            if len(b)>len(s):
                b.pop()
                bd.pop() 
            macdbuy = len(b)
            macdsell = len(s)
            pfmacd= []
            profitmacd = []
            for i in range(len(b)):
                pfmacd.append(((s[i]-b[i])/(b[i]))*money)
                profitmacd.append(((s[i]-b[i])/(b[i])))
            pfmacd = sum(pfmacd)
            # print(len(bd),len(sd))
            calpop.append([cm[0],pfmacd])
    return calpop
        
def binarytournament(p):
    a = random.randint(0,len(p)-1)
    b = random.randint(0,len(p)-1)
    while a==b:
        b = random.randint(0,len(p)-1)
    
    pr1,pr2 = [],[]

    for i in range(len(p)):
        if i == a :
            pr1 = p[i]
        if i == b :  
            pr2 = p[i]
          
    if pr1[1] > pr2[1] :    
        return pr1
    else:
        return pr2

def crossover(p1,p2):
    pcent = random.randint(0,len(p1[0])-1)
    newp1,newp2 = [],[]
    newp1 = p1[0][pcent:]+p2[0][:pcent]
    newp2 = p2[0][pcent:]+p1[0][:pcent]

    p1[0] ,p2[0]= newp1,newp2
    return p1,p2

def mutation(o):
  Mutarate = (1/len(o[0]))*100 # mutation rate = (1/individual size) * 100 

  for i in range(len(o[0])):
    if random.randint(0,100) <= int(Mutarate):
      o[0][i] = random.randint(1,60)

  return o
        


In [24]:
#input
popsize = 50

#variable
generation =1
Maxgen = 200
Elite = 10

#First Generation

pop = initial_population(popsize)
pop = cal_fitness(pop)
pop = sorted(pop,key=lambda l:l[1],reverse=True)
pop

[[[35, 42, 50], 30686.358702107176],
 [[27, 56, 49], 28400.94033914949],
 [[29, 59, 34], 28145.750458024926],
 [[25, 56, 46], 27635.194925256015],
 [[31, 58, 46], 26789.901100399973],
 [[45, 60, 36], 26654.370656124334],
 [[51, 52, 40], 26582.079808331397],
 [[33, 56, 29], 25170.499302546308],
 [[12, 37, 52], 24020.358585139045],
 [[22, 60, 28], 21624.334012884432],
 [[18, 30, 19], 20486.349047917884],
 [[9, 59, 27], 20403.610583897367],
 [[20, 22, 20], 20017.999055095184],
 [[9, 52, 53], 19566.632628756914],
 [[26, 1, 4], 19538.9948407307],
 [[10, 48, 36], 19100.765835080274],
 [[10, 27, 52], 18068.856467144473],
 [[23, 38, 17], 18028.07492125723],
 [[2, 51, 55], 17452.83795369346],
 [[24, 38, 11], 15644.476122681855],
 [[2, 52, 58], 15566.480811930864],
 [[2, 26, 55], 15223.720020382094],
 [[30, 52, 8], 15210.052657450009],
 [[5, 31, 39], 14971.896486226664],
 [[7, 49, 6], 14808.336936304282],
 [[13, 22, 28], 13599.367391474627],
 [[16, 8, 13], 12892.546173250488],
 [[2, 20, 55], 126

In [22]:
random.seed(0)
while generation < Maxgen:

    newpop = []
    for i in range(int(len(pop)*Elite/100)):
        newpop.append(pop[i])
        del pop[i]

    while len(newpop) < popsize:
        parent1,parent2 = binarytournament(pop),binarytournament(pop)
        while parent1 == parent2:
                parent2 = binarytournament(pop)
        # print(parent1)
        parent1,parent2 = crossover(parent1,parent2)
        parent1,parent2 = mutation(parent1),mutation(parent2)
    
        newpop.append(parent1)
        newpop.append(parent2)
    print('Gen =',generation,'Max Fitness =',newpop[0][1],'Best Chomosome = ',newpop[0][0])
    generation += 1
    pop = newpop
    pop = cal_fitness(pop)
    pop = sorted(pop,key=lambda l:l[1],reverse=True)


Gen = 1 Max Fitness = 30060.321502419
36 36
34 34
33 33
68 68
43 43
53 53
103 103
173 173
65 65
66 66
96 96
138 138
173 173
38 38
85 85
81 81
66 66
36 36
45 45
74 74
53 53
29 29
96 96
52 52
38 38
49 49
137 137
33 33
32 32
38 38
66 66
35 35
45 45
35 35
45 45
77 77
96 96
103 103
77 77
47 47
173 173
137 137
67 67
96 96
53 53
103 103
65 65
96 96
138 138
85 85
32 32
Gen = 2 Max Fitness = 30060.321502419
36 36
35 35
38 38
47 47
32 32
87 87
82 82
82 82
76 76
60 60
33 33
60 60
38 38
183 183
343 343
343 343
38 38
183 183
94 94
74 74
82 82
74 74
87 87
52 52
114 114
163 163
38 38
76 76
110 110
85 85
32 32
33 33
114 114
183 183
163 163
116 116
72 72
69 69
85 85
74 74
45 45
255 255
32 32
60 60
52 52
45 45
163 163
82 82
60 60
33 33
82 82
Gen = 3 Max Fitness = 30060.321502419
36 36
38 38
47 47
60 60
60 60
102 102
77 77
40 40
38 38
37 37
65 65
244 244
34 34
360 360
37 37
101 101
32 32
37 37
32 32
122 122
116 116
241 241
34 34
71 71
102 102
102 102
47 47
244 244
92 92
47 47
244 244
40 40
92 92
244 244


KeyboardInterrupt: 

In [140]:
pop

[[[1, 10, 3], 164],
 [[45, 3, 4], 70],
 [[45, 3, 4], 70],
 [[45, 3, 4], 70],
 [[46, 55, 6], 25],
 [[16, 6, 59], 21],
 [[16, 6, 59], 21],
 [[16, 6, 59], 21],
 [[16, 6, 59], 21],
 [[41, 56, 46], 6],
 [[46, 55, 58], 2]]